In [1]:
import pandas as pd
from pathlib import Path
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

data_directory = Path('../data/processed')
try:
    df_final = pd.read_csv(data_directory / 'final_master_dataset_for_prediction.csv')
    print("✅ Tải dữ liệu đã xử lý thành công!")
    print(f"Kích thước dữ liệu: {df_final.shape}")
except FileNotFoundError as e:
    print(f"Lỗi: Không tìm thấy file {e.filename}. Vui lòng chạy notebook data-exploration trước.")
    exit()

✅ Tải dữ liệu đã xử lý thành công!
Kích thước dữ liệu: (3335, 26)


In [2]:
if 'TeamName' in df_final.columns:
    df_final = pd.get_dummies(df_final, columns=['TeamName'], prefix='Team')

# --- 2. Tách Features (X) và Target (y) ---
# Xác định cột mục tiêu (target)
TARGET = 'FinalPosition'

COLS_TO_DROP = [
    TARGET,
    'Year',
    'RaceName',
    'DriverNumber',
    'EventDate',
    'Points'
]

# Tạo X bằng cách loại bỏ các cột không cần thiết
# Dùng errors='ignore' để code không bị lỗi nếu một cột trong list không tồn tại
X = df_final.drop(columns=COLS_TO_DROP, errors='ignore')

# Tạo y từ cột target
y = df_final[TARGET]

# --- 3. Phân chia dữ liệu theo thời gian ---
# Xác định năm cuối cùng làm dữ liệu kiểm tra (test)
test_year = df_final['Year'].max()
print(f"Năm được chọn làm bộ dữ liệu test: {int(test_year)}")

# Tạo điều kiện lọc (boolean mask) cho tập train và test
train_mask = (df_final['Year'] < test_year)
test_mask = (df_final['Year'] == test_year)

# Áp dụng điều kiện lọc để tạo các tập dữ liệu
X_train, X_test = X[train_mask], X[test_mask]
y_train, y_test = y[train_mask], y[test_mask]


print(f"Huấn luyện trên {X_train.shape[0]} mẫu, bao gồm {X_train.shape[1]} features.")
print(f"Kiểm tra trên {X_test.shape[0]} mẫu, bao gồm {X_test.shape[1]} features.")

Năm được chọn làm bộ dữ liệu test: 2025
Huấn luyện trên 2976 mẫu, bao gồm 38 features.
Kiểm tra trên 359 mẫu, bao gồm 38 features.


In [10]:
model = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=7,
    early_stopping_rounds=50,
    random_state=42
)

model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

predictions = model.predict(X_test)
mae = mean_absolute_error(y_test, predictions)
print(f"Sai số dự đoán trung bình (MAE): {mae:.2f} vị trí")

Sai số dự đoán trung bình (MAE): 2.41 vị trí
